<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/280_EPO_AgentDef.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experimentation Portfolio Orchestrator Agent

In [ ]:
"""Experimentation Portfolio Orchestrator Agent

This agent manages the full lifecycle of experiments across an organization:
- Loads and analyzes experiment portfolio data
- Generates recommendations (scale, iterate, retire)
- Creates comprehensive portfolio reports
"""

In [ ]:
# ============================================================================
# Experimentation Portfolio Orchestrator Agent
# ============================================================================

class ExperimentationPortfolioOrchestratorState(TypedDict, total=False):
    """State for Experimentation Portfolio Orchestrator Agent"""

    # Input fields
    experiment_id: Optional[str]            # Specific experiment to analyze (None = analyze all)

    # Goal & Planning fields (MVP: Fixed goal, template-based plan)
    goal: Dict[str, Any]                     # Goal definition (from goal_node)
    plan: List[Dict[str, Any]]              # Execution plan (from planning_node)

    # Data Ingestion
    portfolio: List[Dict[str, Any]]         # Loaded experiment portfolio data
    # Structure per experiment:
    # {
    #   "experiment_id": "E001",
    #   "experiment_name": "AI Email Drafting for Sales",
    #   "domain": "sales",
    #   "owner": "growth_team",
    #   "status": "completed" | "running" | "planned",
    #   "start_date": "2024-10-01",
    #   "end_date": "2024-10-14" | null
    # }

    experiment_definitions: List[Dict[str, Any]]  # Loaded experiment definitions
    # Structure per experiment:
    # {
    #   "experiment_id": "E001",
    #   "hypothesis": "...",
    #   "variants": ["control", "ai_drafted"],
    #   "primary_metric": "reply_rate",
    #   "secondary_metrics": ["meeting_booked_rate"],
    #   "success_criteria": "...",
    #   "owner": "growth_team",
    #   "status": "completed"
    # }

    experiment_metrics: List[Dict[str, Any]]  # Loaded experiment metrics
    # Structure per variant:
    # {
    #   "experiment_id": "E001",
    #   "variant": "control",
    #   "reply_rate": 0.18,
    #   "meeting_booked_rate": 0.05,
    #   "sample_size": 500
    # }

    experiment_analysis: List[Dict[str, Any]]  # Loaded experiment analysis results
    # Structure per experiment:
    # {
    #   "experiment_id": "E001",
    #   "primary_metric": "reply_rate",
    #   "control_value": 0.18,
    #   "treatment_value": 0.26,
    #   "absolute_lift": 0.08,
    #   "relative_lift_percent": 44.4,
    #   "direction": "positive",
    #   "confidence": "medium",
    #   "summary": "..."
    # }

    experiment_decisions: List[Dict[str, Any]]  # Loaded experiment decisions
    # Structure per experiment:
    # {
    #   "experiment_id": "E001",
    #   "decision": "scale" | "iterate" | "retire" | "do_not_start",
    #   "rationale": "...",
    #   "recommended_action": "...",
    #   "owner": "growth_team",
    #   "decision_date": "2024-10-20"
    # }

    # Data Lookups (for fast access)
    portfolio_lookup: Dict[str, Dict[str, Any]]  # experiment_id -> portfolio entry
    definitions_lookup: Dict[str, Dict[str, Any]]  # experiment_id -> definition
    metrics_lookup: Dict[str, List[Dict[str, Any]]]  # experiment_id -> list of variant metrics
    analysis_lookup: Dict[str, Dict[str, Any]]  # experiment_id -> analysis result
    decisions_lookup: Dict[str, Dict[str, Any]]  # experiment_id -> decision

    # Portfolio Analysis
    analyzed_experiments: List[Dict[str, Any]]  # Experiments with complete analysis
    # Structure per experiment:
    # {
    #   "experiment_id": "E001",
    #   "status": "completed",
    #   "has_metrics": True,
    #   "has_analysis": True,
    #   "has_decision": True,
    #   "analysis_status": "complete" | "partial" | "missing",
    #   "needs_analysis": False,
    #   "needs_decision": False
    # }

    portfolio_summary: Dict[str, Any]  # Portfolio-level summary metrics
    # Structure:
    # {
    #   "total_experiments": 3,
    #   "completed_count": 1,
    #   "running_count": 1,
    #   "planned_count": 1,
    #   "experiments_with_analysis": 2,
    #   "experiments_with_decisions": 2,
    #   "experiments_needing_analysis": 0,
    #   "experiments_needing_decisions": 0,
    #   "domains": ["sales", "customer_support", "hr"],
    #   "total_sample_size": 1630,
    #   "average_lift_percent": 6.7
    # }

    # Experiment Analysis (calculated/updated)
    calculated_analyses: List[Dict[str, Any]]  # Newly calculated analyses
    # Same structure as experiment_analysis

    # Decision Generation
    generated_decisions: List[Dict[str, Any]]  # Newly generated decisions
    # Same structure as experiment_decisions

    # Portfolio Insights
    portfolio_insights: List[Dict[str, Any]]  # High-level insights across portfolio
    # Structure per insight:
    # {
    #   "type": "trend" | "risk" | "opportunity" | "recommendation",
    #   "title": "...",
    #   "description": "...",
    #   "experiments": ["E001", "E002"],
    #   "priority": "high" | "medium" | "low"
    # }

    # Output
    portfolio_report: str                    # Final markdown report
    report_file_path: Optional[str]          # Path to saved report file

    # Metadata
    errors: List[str]                       # Any errors encountered
    processing_time: Optional[float]        # Time taken to process


@dataclass
class ExperimentationPortfolioOrchestratorConfig:
    """Configuration for Experimentation Portfolio Orchestrator Agent"""
    llm_model: str = os.getenv("LLM_MODEL", "gpt-4o-mini")
    temperature: float = 0.3
    reports_dir: str = "output/experimentation_portfolio_reports"  # Where to save reports

    # Data file paths
    data_dir: str = "data"
    portfolio_file: str = "experiment_portfolio.json"
    definitions_file: str = "experiment_definitions.json"
    metrics_file: str = "experiment_metrics.json"
    analysis_file: str = "experiment_analysis.json"
    decisions_file: str = "experiment_decisions.json"

    # Analysis Settings
    min_sample_size: int = 100  # Minimum sample size for statistical validity
    confidence_threshold: float = 0.05  # p-value threshold for significance

    # Decision Criteria (MVP: Rule-based)
    scale_threshold_lift: float = 20.0  # Minimum % lift to recommend scaling
    iterate_threshold_lift: float = 5.0  # Minimum % lift to recommend iterating
    retire_threshold_lift: float = -10.0  # Below this % change, recommend retiring

    # Portfolio Analysis
    enable_portfolio_insights: bool = True  # Generate cross-experiment insights

    # Toolshed Integration (placeholders for future use)
    enable_progress_tracking: bool = False  # Placeholder for toolshed.progress
    enable_kpi_tracking: bool = False  # Placeholder for toolshed.kpi
    enable_hitl: bool = False  # Placeholder for toolshed.hitl


# Data Loading Utilities

In [ ]:
"""Data Loading Utilities

Load experiment data from JSON files and build lookup dictionaries.
"""

import json
from pathlib import Path
from typing import Dict, List, Any, Optional


def load_json_file(file_path: str) -> List[Dict[str, Any]]:
    """Load JSON file and return list of dictionaries."""
    path = Path(file_path)
    if not path.exists():
        raise FileNotFoundError(f"Data file not found: {file_path}")

    with open(path, 'r') as f:
        data = json.load(f)

    if not isinstance(data, list):
        raise ValueError(f"Expected list, got {type(data)} for {file_path}")

    return data


def load_portfolio_data(data_dir: str, filename: str) -> List[Dict[str, Any]]:
    """Load experiment portfolio data."""
    file_path = Path(data_dir) / filename
    return load_json_file(str(file_path))


def load_definitions_data(data_dir: str, filename: str) -> List[Dict[str, Any]]:
    """Load experiment definitions data."""
    file_path = Path(data_dir) / filename
    return load_json_file(str(file_path))


def load_metrics_data(data_dir: str, filename: str) -> List[Dict[str, Any]]:
    """Load experiment metrics data."""
    file_path = Path(data_dir) / filename
    return load_json_file(str(file_path))


def load_analysis_data(data_dir: str, filename: str) -> List[Dict[str, Any]]:
    """Load experiment analysis data."""
    file_path = Path(data_dir) / filename
    return load_json_file(str(file_path))


def load_decisions_data(data_dir: str, filename: str) -> List[Dict[str, Any]]:
    """Load experiment decisions data."""
    file_path = Path(data_dir) / filename
    return load_json_file(str(file_path))


def build_portfolio_lookup(portfolio: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """Build lookup dictionary: experiment_id -> portfolio entry."""
    return {exp["experiment_id"]: exp for exp in portfolio}


def build_definitions_lookup(definitions: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """Build lookup dictionary: experiment_id -> definition."""
    return {exp["experiment_id"]: exp for exp in definitions}


def build_metrics_lookup(metrics: List[Dict[str, Any]]) -> Dict[str, List[Dict[str, Any]]]:
    """Build lookup dictionary: experiment_id -> list of variant metrics."""
    lookup: Dict[str, List[Dict[str, Any]]] = {}
    for metric in metrics:
        exp_id = metric["experiment_id"]
        if exp_id not in lookup:
            lookup[exp_id] = []
        lookup[exp_id].append(metric)
    return lookup


def build_analysis_lookup(analysis: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """Build lookup dictionary: experiment_id -> analysis result."""
    return {exp["experiment_id"]: exp for exp in analysis}


def build_decisions_lookup(decisions: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """Build lookup dictionary: experiment_id -> decision."""
    return {exp["experiment_id"]: exp for exp in decisions}

